# 递归
- https://blog.csdn.net/liuergo/article/details/104345089
- 1、明确递归函数的输入输出功能，将递归函数看作一个黑盒。
- 2、明确递归结束的条件
- 3、缩小递归范围，获得等价条件（要么获得递推式，要么寻找缩小递归范围的办法，使递归不断趋近递归结束条件）

In [3]:
# 例题1:计算阶乘
def jiecheng(n):
    if n <= 2:
        return n
    else:
        return n * jiecheng(n-1)

def jiecheng_dp(n):
    r = [1]
    for i in range(1, n+1):
        r.append(r[i-1]*i)
    return r[n]

jiecheng_dp(4)
jiecheng(4)

24

# 动态规划
- 1、探索状态转移方程（递推公式）
- 2、base case基础问题解决

In [ ]:
# 普通的递归方式解决问题，会重复计算子问题，导致计算时间过长
def febnacci(n):
    if n == 1 or n == 2:
        return 1
    else:
        return febnacci(n-1) + febnacci(n-2)

# 动态规划就是解决：递归+重复子问题

## 1、钢条切割问题

In [1]:
# 例题1:如何切割一个钢条，使其总收益最大（不同的长度有不同的价格，越长价格越高）

# 最优子结构：问题的最优解由相关子问题的最优解组合而成，这些子问题可以独立求解。
# 找出递推式！

# 钢条切割问题，递推式可以简化为左边不切右边切。

In [4]:
# 解决方法1:自顶向下（rn，rn-1……）递归方法
price = [0,1,5,8,9,10,17,17,20,24,30]
def cut_rod_recurision_1(p, n):
    if n == 0:
        return 0
    else:
        res = p[n]
        for i in range(1, n):
            res = max(res, cut_rod_recurision_1(p, i)+cut_rod_recurision_1(p, n-i))
        return res

def cut_rod_recurision_2(p, n):
    if n == 0:
        return 0
    else:
        res = 0
        for i in range(1, n+1):
            res = max(res, p[i]+cut_rod_recurision_2(p, n-i))
        return res

cut_rod_recurision_2(price, 9)

25

In [6]:
# 解决方法2:自底向上，动态规划方式解决（从r0开始计算并保存结果）
def cut_rod_dp(p, n):
    r = [0]
    for i in range(1, n+1):  #i是从1到n遍历长度
        res = 0
        for j in range(1, i+1):  #j是切一刀后左边的长度
            res = max(res, p[j]+r[i-j]) #左边不切右边切（i-j一定是小于i的）
        r.append(res)
    return r[n]

cut_rod_dp(price, 9)

25

In [12]:
# 如何得到最优值，并且同时得到切割方案
def cut_rod_extend(p, n):
    r = [0]
    s = [0]
    for i in range(1, n+1):
        res_r = 0
        res_s = 0
        for j in range(1, i+1):
            if p[j]+r[i-j] > res_r:
                res_r = p[j]+r[i-j]
                res_s = j  #如果切后更优，就保存左边的长度
        r.append(res_r)
        s.append(res_s)  #保存的每一种长度最优解下左边的长度
    return r[n], s

def cut_rod_solution(p, n):
    r, s = cut_rod_extend(p, n)
    ans = []
    while n > 0:
        ans.append(s[n])
        n -= s[n]
    return r, ans

cut_rod_solution(price, 9)

(25, [3, 6])

In [ ]:
# 什么问题可以用动态规划
# 1、最优子问题
# 原问题的最优解中设计多少个子问题
# 在确定最优解使用哪些子问题时，需要考虑多少选择
# （贪心算法相当于动态规划的一个特例）
# 2、 重叠子问题

## 2、字符串相关问题
解决两个字符串的动态规划问题，一般都是用两个指针 i,j 分别指向两个字符串的最后，然后一步步往前走，缩小问题的规模。

In [7]:
# 例题2:最长公共子序列
# （一个序列的子序列是在该序列删去若干元素后得到的序列，顺序不能变，不要求连续）
# 最长公共子序列LCS问题：给定两个序列X和Y，求X和Y长度最大的公共子序列
# 应用场景：字符串相似度比对、基因工程比对

def lcs_length(x, y):
    m = len(x)
    n = len(y)
    c = [[0 for _ in range(n+1)] for _ in range(m+1)]  # c存储当前的LCS的长度，+1是多加一个空串
    for i in range(1, m+1):
        for j in range(1, n+1):
            if x[i-1] == y[j-1]:  # 注意字符串的下标是从0开始的
                c[i][j] = c[i-1][j-1] + 1
            else:
                c[i][j] = max(c[i-1][j], c[i][j-1])
    for _ in c:
        print(_)
    return c[m][n]

def lcs(x, y):
    m = len(x)
    n = len(y)
    c = [[0 for _ in range(n+1)] for _ in range(m+1)]
    b = [[0 for _ in range(n+1)] for _ in range(m+1)]  # 1是↖️，2是⬆️，3是⬅️，0是第一行第一列无方向
    for i in range(1, m+1):
        for j in range(1, n+1):
            if x[i-1] == y[j-1]:  # 注意字符串的下标是从0开始的
                c[i][j] = c[i-1][j-1] + 1
                b[i][j] = 1
            elif c[i-1][j] > c[i][j-1]:
                c[i][j] = c[i-1][j]
                b[i][j] = 2
            else:
                c[i][j] = c[i][j-1]
                b[i][j] = 3
    return c[m][n], b

def lcs_trackback(x, y):
    c, b = lcs(x, y)
    i = len(x)
    j = len(y)
    res = []
    while i > 0 and j > 0:
        if b[i][j] == 1:  #来自左上，匹配
            res.append(x[i-1])
            i -= 1
            j -= 1
        elif b[i][j] == 2:  #来自上方，不匹配
            i -= 1
        else:  # 来自左方，也不匹配
            j -= 1
    return "".join(reversed(res))

# lcs_length("ABCBDAB", "BDCABA")
lcs_trackback("ABCBDAB", "BDCABA")

'BDAB'

In [19]:
# 例题3:最小编辑距离
# https://zhuanlan.zhihu.com/p/80682302
# 用最少的步骤完成字符串的修改

def solution_recurision(s1, s2):
    """递归方法"""
    def bianji(i, j):
        if i == -1: return j + 1
        if j == -1: return i + 1
        if s1[i] == s2[j]:
            return bianji(i-1, j-1)
        else:
            return min(
                bianji(i-1, j)+1,  #插入
                bianji(i, j-1)+1,  #删除
                bianji(i-1, j-1)+1  #替换
            )
    return bianji(len(s1)-1, len(s2)-1)

def solution_dp(s1, s2):
    """动态规划方法"""
    m = len(s1)
    n = len(s2)
    table = [[0 for j in range(n+1)] for i in range(m+1)]
    choice = [[0 for j in range(n+1)] for i in range(m+1)]

    # 先考虑特殊情况，即空字符串与非空字符串的最小编辑距离
    for i in range(m+1):
        table[i][0] = i;
    for j in range(n+1):
        table[0][j] = j;

    # 填补table中的其他地方
    for i in range(1, m+1):
        for j in range(1, n+1):
            if s1[i-1] == s2[j-1]:
                table[i][j] = table[i-1][j-1]
            else:
                table[i][j] = min(
                    table[i-1][j]+1,
                    table[i][j-1]+1,
                    table[i-1][j-1]+1
                )
    for _ in table:
        print(_)
    return table[m][n]

# solution_recurision("rad", "apple")
solution_dp("rad", "apple")

[0, 1, 2, 3, 4, 5]
[1, 1, 2, 3, 4, 5]
[2, 1, 2, 3, 4, 5]
[3, 2, 2, 3, 4, 5]


5

## 3、股票问题

In [35]:
# 例题：买卖股票的最佳时机
# https://leetcode.cn/problems/best-time-to-buy-and-sell-stock/solution/yi-ge-fang-fa-tuan-mie-6-dao-gu-piao-wen-ti-by-l-3/

def baoli(s):
    """暴力解法，力扣上说超出时间限制"""
    res = 0
    for i in range(len(s)):
        for j in range(i+1, len(s)):
            tmp = s[j]-s[i]
            if tmp >= 0 and tmp > res:
                res = tmp
    return res
            
# li = [7,1,5,3,6,4]
# baoli(li)

# 动态规划方法
# dp[i][k][0]表示当天i在k次交易的限制下没有持有股票的最后状态
# dp[i][k][]表示当天i在k次交易的限制下有持有股票的最后状态

# base case：
# dp[-1][k][0] = dp[i][0][0] = 0
# dp[-1][k][1] = dp[i][0][1] = -infinity
# 状态转移方程：
# dp[i][k][0] = max(dp[i-1][k][0], dp[i-1][k][1] + prices[i])
# dp[i][k][1] = max(dp[i-1][k][1], dp[i-1][k-1][0] - prices[i])

def ticket_dp(s, k):
    n = len(s)
    dp = [[0 for m in range(2)]for i in range(n)]

    for i in range(0, n):
        if i-1 == -1:
            dp[i][0] = 0
            dp[i][1] = -s[i]
            continue
        dp[i][0] = max(dp[i-1][0], dp[i-1][1]+s[i])
        dp[i][1] = max(dp[i-1][1], -s[i])
    print(dp)
    return dp[n-1][0]

s = [7,1,5,3,6,4]
k = 1
ticket_dp(s, k)

[[0, -7], [0, -1], [4, -1], [4, -1], [5, -1], [5, -1]]


5

## 4、背包问题
https://blog.csdn.net/stack_queue/article/details/53544109

### 第一讲 01背包问题
这是最基本的背包问题，每个物品最多只能放一次。

### 第二讲 完全背包问题
第二个基本的背包问题模型，每种物品可以放无限多次。

### 第三讲 多重背包问题
每种物品有一个固定的次数上限。

### 第四讲 混合三种背包问题
将前面三种简单的问题叠加成较复杂的问题。

### 第五讲 二维费用的背包问题
一个简单的常见扩展。

### 第六讲 分组的背包问题
一种题目类型，也是一个有用的模型。后两节的基础。

### 第七讲 有依赖的背包问题
另一种给物品的选取加上限制的方法。

### 第八讲 泛化物品
我自己关于背包问题的思考成果，有一点抽象。